In [ ]:
!pip install -q transformers datasets sentencepiece accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.2 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload `omp_train.jsonl` and `omp_val.jsonl`

Saving omp_train.jsonl to omp_train.jsonl
Saving omp_val.jsonl to omp_val.jsonl


In [ ]:
import json

def load_jsonl(path):
    with open(path, "r") as f:
        return [json.loads(line) for line in f]

train_data = load_jsonl("omp_train.jsonl")
val_data = load_jsonl("omp_val.jsonl")

print(f"Train: {len(train_data)}, Validation: {len(val_data)}")
print("Sample:", train_data[0])

✅ Train: 30305, Validation: 3368
🔹 Sample: {'input': 'atomic codegen flush for ir masked parser sections target', 'output': 'llvm/lib/Target/AArch64/AArch64ISelLowering.cpp::shouldSinkVectorOfPtrs'}


In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:

max_input_len = 32
max_output_len = 256

def preprocess(example):
    model_input = tokenizer(example['input'], max_length=max_input_len, truncation=True, padding="max_length")
    label = tokenizer(example['output'], max_length=max_output_len, truncation=True, padding="max_length")

    model_input['labels'] = label['input_ids']
    return model_input

train_dataset = train_dataset.map(preprocess, remove_columns=['input', 'output'])
val_dataset = val_dataset.map(preprocess, remove_columns=['input', 'output'])


Map:   0%|          | 0/30305 [00:00<?, ? examples/s]

Map:   0%|          | 0/3368 [00:00<?, ? examples/s]

In [ ]:
import transformers
print(transformers.__version__)

4.52.4


In [ ]:

from transformers import TrainingArguments, Trainer
import os
os.environ["WANDB_DISABLED"] = "true"
args = TrainingArguments(
    output_dir="./omp_model",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    learning_rate=5e-4,
    weight_decay=0.01,
    eval_strategy="epoch",  # Changed from evaluation_strategy
    save_strategy="epoch",
    logging_dir="./logs",
    push_to_hub=False,
    fp16=False  # Use True if GPU supports it
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-9-1770201057>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:

trainer.train()


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.014300,0.010312
2,0.006800,0.006810
3,0.004200,0.005038
4,0.003800,0.004191
5,0.002400,0.003973


TrainOutput(global_step=18945, training_loss=0.010535486828654761, metrics={'train_runtime': 4364.0556, 'train_samples_per_second': 34.721, 'train_steps_per_second': 4.341, 'total_flos': 1281729154252800.0, 'train_loss': 0.010535486828654761, 'epoch': 5.0})

In [ ]:
def predict(prompt, max_len=256):
    device = model.device  # Detect model device (cuda or cpu)
    input_ids = tokenizer(prompt, return_tensors="pt", padding=True).input_ids.to(device)
    output_ids = model.generate(input_ids, max_length=max_len, num_beams=5, early_stopping=True)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)
prompt = "omp barrier"
print(f"Prompt: {prompt}\n Predicted Files/Functions:\n{predict(prompt)}")


🧠 Prompt: omp barrier
🎯 Predicted Files/Functions:
mlir/lib/Conversion/NVGPUToNVVM/NVGPUToNVVM.cpp::isMbarrierShared, mlir/lib/Conversion/NVGPUToNVVM/NVGPUToNVVM.cpp::memRefDescriptor, mlir/lib/Dialect/NVGPU/TransformOps/NVGPUTransformOps.cpp::buildTryWaitParity


In [ ]:

model.save_pretrained("omp_t5_model")
tokenizer.save_pretrained("omp_t5_model")

!zip -r omp_t5_model.zip omp_t5_model
files.download("omp_t5_model.zip")


  adding: omp_t5_model/ (stored 0%)
  adding: omp_t5_model/model.safetensors (deflated 8%)
  adding: omp_t5_model/added_tokens.json (deflated 83%)
  adding: omp_t5_model/spiece.model (deflated 48%)
  adding: omp_t5_model/config.json (deflated 62%)
  adding: omp_t5_model/special_tokens_map.json (deflated 85%)
  adding: omp_t5_model/generation_config.json (deflated 29%)
  adding: omp_t5_model/tokenizer_config.json (deflated 94%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>